# Fusion

Attempt to apply the approach used by [Nir Diamant](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/fusion_retrieval.ipynb) to the ragsc problem.

## Strategy

Consider each cluster as a "document".  Using a random sample of the cluster data and associated embeddings, create a vector database
using FAISS or Chroma.  At the same time, use Lucene to create an index for the "documents".  Score matches on both semantic (vector) and keyword (BM25) and combine the scores to see if we can get more success matching to clusters.

In [41]:
#
# import libraries
#
import pandas as pd
from pathlib import Path
from rank_bm25 import BM25Okapi
import numpy as np
from itertools import chain
from functools import partial, reduce
from typing import Union
from loguru import logger

In [2]:
#
# set constants
#
input_path = Path("../results")
output_path = Path("../results")
training_fraction = 0.5

In [3]:
#
# load the data along with embeddings
#
master_df = pd.read_csv(input_path / Path("ragsc_00_all_large.csv"))
master_n_cells = master_df.shape[0]

train_df = master_df.sample(frac=training_fraction)
test_df= master_df.drop(train_df.index) #.sample(frac=training_fraction) 
print(f"total rows: {master_df.shape[0]}")
print(f"training set has {train_df.shape[0]} rows")
print(f"test set has {test_df.shape[0]} rows")

total rows: 9370
training set has 4685 rows
test set has 4685 rows


In [4]:
for cluster in test_df.groupby('cluster'):
    print(cluster[0], cluster[1].shape[0])

0 624
1 527
2 380
3 353
4 367
5 327
6 276
7 270
8 240
9 213
10 201
11 190
12 147
13 142
14 107
15 93
16 102
17 78
18 48


In [5]:
def get_gene_bags(df: pd.DataFrame, max_genes:int, sort_by_cluster_names=True) -> dict:
    """
    Produces "bags of words" for each cluster to use as documents in BM25 analysis.
    
    Returns a dictionary with cluster name as the keys and a list of gene names as the values.
    """
    clusters = df.groupby("cluster", sort=False)
    word_dict = {}
    for cluster in clusters:
        # each cluster is a tuple (cluster name, cluster dataframe)
        words = []
        cluster_df = cluster[1] # the dataframe
        # convert each signature into a list of string
        word_series = cluster_df.signature.apply(lambda x: x.split(" "))
        # create a bag of words based containing the gene names for this cluster
        for sig in word_series:
            # retain only max_genes gene names to add to the bag of words
            words.extend(sig[:max_genes]) 
        word_dict[cluster[0]] = words
    if sort_by_cluster_names:
        word_dict = {k: word_dict[k] for k in sorted(word_dict)}
    return word_dict


In [7]:
#
# chunking
#
def chunk(s:Union[str,list], size:int, step=1) -> list[str]:
    """
    Takes a string or list of strings and creates a list of overlapping chunks of a given size.

    Args
        size: The number of words (gene names) in each chunk.
        step: The number of words to advance before the next chunk (defaults to 1).
    Returns
        A list of strings representing the chunks.
    """
    if isinstance(s,str):
        a = s.split()
    else:
        a = s
    results = []
    max = len(a)
    for i in range(max):
        if i+size < max:
            results.append(" ".join(a[i:i+size]))
        else:
            results.append(" ".join(a[i:]))
        i += step
    return results
    
# chunks = chunk("this is a test of the splitter", 3)
# print(chunks)
chunk2 = partial(chunk, size=2)

In [8]:
chunkn = partial(chunk, size=2)

In [9]:
word_dict = get_gene_bags(train_df,max_genes=120)

#
# create index from the cluster "documents" which are stored in word_dict
#
docs = [chunk2(" ".join(x)) for x in word_dict.values()]
bm25_index = BM25Okapi(docs)


In [10]:
def get_score(bm25, gene_list, max_genes=25, normalized=True) -> list[float]:
    """
    Returns a list containing the scores for a particular list of genes
    """
    query = chunk2(gene_list)[:max_genes]
    scores = bm25.get_scores(query)
    if normalized:
        scores = (scores - np.min(scores))/(np.max(scores)-np.min(scores))
    return scores

In [11]:
def create_score_column(df:pd.DataFrame, bm25, max_genes) -> pd.DataFrame:
    """
    Add a column to the provided dataframe containing the BM25 scores.

    Args:
        df - the dataframe whose signatures will be used to generate the scores
        bm25 - the index to use fo comparison
        max_genes - the maximum number of genes to include from each signature

    Returns a reference to the original dataframe
    """
    df['scores'] = df.signature.apply(lambda x: get_score(bm25, x, max_genes))
    return df

In [12]:
df_test = create_score_column(test_df, bm25_index, 120)

In [13]:
n=121
cluster = test_df.cluster.iloc[n]
scores = test_df.scores.iloc[n]
rating = scores[cluster]
print(cluster, scores, rating)

0 [1.         0.91118707 0.71737601 0.63492394 0.66268237 0.49215485
 0.32284543 0.42814737 0.27423234 0.42334994 0.16279759 0.
 0.27388819 0.2822286  0.01755665 0.15366833 0.12148767 0.12874504
 0.10166728] 1.0


In [14]:
row = 25
clusters = df_test.groupby('cluster')
for cluster in clusters:
    # print(cluster[1].shape)
    local_df = cluster[1]
    no = cluster[0]
    if no> 0:
        bad = no-1
    else:
        bad = no+1
    scores = local_df.scores.iloc[row]
    # print(scores)
    print(f"{no:5} {scores[no]:8.2f} {scores[bad]:8.2f}")


    0     1.00     0.97
    1     1.00     0.80
    2     0.71     0.69
    3     1.00     0.91
    4     1.00     0.33
    5     1.00     0.20
    6     1.00     0.65
    7     0.97     0.49
    8     1.00     0.23
    9     1.00     0.27
   10     1.00     0.12
   11     1.00     0.52
   12     1.00     0.19
   13     1.00     0.24
   14     1.00     0.25
   15     1.00     0.12
   16     0.45     0.41
   17     0.25     0.16
   18     0.37     0.20


In [15]:
#
# explore the effect of sample size on mean score
#
print(test_df.shape[0])
print()
sum=0
n_clusters = 19
count=0
# test_df['scores_sum'] = test_df.scores.apply(lambda x: np.sum(x))
for cluster in test_df.groupby('cluster'):
    cluster_no = cluster[0]
    cluster_df = cluster[1]
    cluster_df['predicted_score'] = cluster_df.scores.apply(lambda x: x[cluster_no])
    # assume score is normalized
    avg_score_for_cluster = cluster_df.predicted_score.sum() / cluster_df.shape[0]
    print(f"Cluster: {cluster_no:02}: {avg_score_for_cluster:8.3f} ({cluster_df.shape[0]:02})")

4685

Cluster: 00:    0.993 (624)
Cluster: 01:    0.962 (527)
Cluster: 02:    0.875 (380)
Cluster: 03:    0.898 (353)
Cluster: 04:    0.879 (367)
Cluster: 05:    0.963 (327)
Cluster: 06:    0.988 (276)
Cluster: 07:    0.784 (270)
Cluster: 08:    0.957 (240)
Cluster: 09:    0.992 (213)
Cluster: 10:    0.965 (201)
Cluster: 11:    0.985 (190)
Cluster: 12:    0.639 (147)
Cluster: 13:    0.830 (142)
Cluster: 14:    0.922 (107)
Cluster: 15:    0.811 (93)
Cluster: 16:    0.635 (102)
Cluster: 17:    0.391 (78)
Cluster: 18:    0.386 (48)


In [16]:
#
# explore the effect of training set size on mean score
#
for cluster in train_df.groupby('cluster'):
    cluster_no = cluster[0]
    cluster_df = cluster[1]
    print(f"Cluster: {cluster_no:02}:({cluster_df.shape[0]:02})")

Cluster: 00:(608)
Cluster: 01:(547)
Cluster: 02:(411)
Cluster: 03:(374)
Cluster: 04:(328)
Cluster: 05:(288)
Cluster: 06:(295)
Cluster: 07:(271)
Cluster: 08:(251)
Cluster: 09:(193)
Cluster: 10:(197)
Cluster: 11:(178)
Cluster: 12:(167)
Cluster: 13:(151)
Cluster: 14:(113)
Cluster: 15:(108)
Cluster: 16:(97)
Cluster: 17:(70)
Cluster: 18:(38)


In [18]:
#
# save intermediate results
#
train_df.to_csv("data/train.csv")
test_df.to_csv("data/test.csv")

## Vector database strategy

Each cluster is a text document.
Each cell signature is a sentence.
Need to chunk the cluster documents and restrict sentences to the highest expression genes. A reasonable cut point is 120 genes based on the BM25 analysis, which showed plateauing in the matches at around this number of "words".

In [29]:
#
# docs contains the chunked gene names by cluster
#

total = reduce(lambda x,y: x+y, [len(x) for x in docs],0)
print(total)

562200


In [45]:
#
# given that current form of docs is prohibitively large (n=562200 chunks),
# will use current embeddings as a first attempt
#

In [62]:
import chromadb
import json

In [63]:
def store_embeddings(
    collection: chromadb.Collection,
    df: pd.DataFrame,
    min_item=0,
    max_item=-1,
    embeddings_column: str = "embeddings",
    docs_column: str = "cluster",
) -> int:
    """
    Stores embeddings in the provided ChromaDB collection.

    Args
    collection: the collection to receive the data
    df : the dataframe from which the data is derived
    min_item: the minimum row number to use
    max_item: the maximum row number to use, defaults to -1 (all rows)
    embeddings_column: the column containing the embeddings, defaults to "embeddings"
    docs_column: the column containing the document name, defaults to "cluster"

    Rerturns the number of embeddings added to the database
    """
    if max_item == -1:
        max_item = df.shape[0]
    if max_item <= min_item:
        logger.error("max_item must be greater than min_item")
        return 0
    docs = [] # clusters
    embeds = [] # embeddings
    ids = [] # cell ids
    for i in range(min_item, max_item):
        docs.append(str(df[docs_column].iloc[i]))
        embeds.append(json.loads(df[embeddings_column].iloc[i]))
        ids.append(str(df.index[i]))
    try:
        collection.add(documents=docs, embeddings=embeds, ids=ids)
    except Exception as e:
        logger.error("unable to load data into database")
        logger.exception(e)
        return 0
    else:
        return max_item - min_item

In [64]:
def initialize_database(collection_name: str = "ragsc") -> chromadb.Collection:
    client = chromadb.Client()
    try:
        c = client.get_collection(collection_name)
        client.delete_collection(collection_name)
    except ValueError:
        pass
    c = client.create_collection(collection_name)
    return c

In [65]:
def setup_database(df: pd.DataFrame) -> chromadb.Collection:
    """
    creates an in memory ChromaDB collection  based on the data in the 
    provided dataframe.
    """
    collection = initialize_database()
    df = df[~df.signature.isnull()]  # clean any empty signatures
    store_embeddings(collection, df)
    return collection

In [173]:
def test_embeddings(embeddings:str, collection:chromadb.Collection, n_results=100):
    results = collection.query(
        query_embeddings=[json.loads(embeddings)],
        n_results=n_results,
        include=["documents","distances"])
    return results

In [170]:
#
# need to test results based on cluster orientation
#
def test_item(df: pd.DataFrame, row: int, collection: chromadb.Collection, n_results=100):
    # print(f"Original cluster: {df.cluster.iloc[row]}")
    results = collection.query(
        query_embeddings=[json.loads(df.embeddings.iloc[row])],
        n_results=n_results,
        include=["documents","distances"]
    )
    # print(results)
    # return zip(results['documents'],results['distances'])
    return results


In [164]:
#
# first attempt
#
coll = setup_database(train_df)
results = test_item(test_df,0,coll)


In [165]:
def distance_score(data: list[float], offset = 0.01)->float:
    if len(data) == 0:
        return 0 
    a = np.array(data)
    a = np.log10(1.0/(a+offset))
    return a.sum()
        

In [171]:
def distance_score_per_row(embeddings:str, coll, n_results=100, max_clusters=19):
    results = test_embeddings(embeddings,coll)
    pairs = list(zip(results['documents'][0],results['distances'][0]))
    table ={}
    for k in range(max_cluster): 
        table[k] = []
    for item in pairs:
        table[int(item[0])].append(item[1])
    scores = []
    for k in table:
        scores.append(distance_score(table[k]))
    return scores

In [ ]:
def apply_distance_score(df: pd.DataFrame, coll: chromadb.Collection, n_results=100) -> None:
    """
    Calculates distance_score on a row-wise basis, storing the results
    in column "d_score" and normalized (x-min/max-min) in "n_score".
    
    This works in-place, modifying the input dataframe by adding two columns.
    """
    # first create columns of lists to store the results
    df['d_score'] = [[] for i in range(df.shape[0])]
    df['n_score'] = [[] for i in range(df.shape[0])]

    # now calculate the scores and normalized scores
    df.d_score = df.embeddings.apply(lambda x: distance_score_per_row(x, coll))
    df.n_score = df.d_score.apply(lambda x: (x-np.min(x))/(np.max(x) - np.min(x)))

In [ ]:
#
# calculate the distance scores
#
apply_distance_score(test_df, coll)

In [ ]:
test_df.head(2)

In [228]:
#
# calculate the combined index
#
alpha = 0 # alpha is the proportion to assign to each score

df_test['overall'] = df_test.scores * alpha + (1 - alpha) * df_test.n_score

In [229]:
clusters = df_test.groupby('cluster')
for cluster in clusters:
    cluster_no = cluster[0]
    cluster_df = cluster[1]
    cluster_df['accuracy_score'] = cluster_df.overall.apply(lambda x: x[cluster_no])
    # assume score is normalized
    avg_score_for_cluster = cluster_df.accuracy_score.sum() / cluster_df.shape[0]
    print(f"Cluster: {cluster_no:02}: {avg_score_for_cluster:8.3f} ({cluster_df.shape[0]:02})")

Cluster: 00:    0.757 (624)
Cluster: 01:    0.998 (527)
Cluster: 02:    0.626 (380)
Cluster: 03:    0.672 (353)
Cluster: 04:    0.614 (367)
Cluster: 05:    0.702 (327)
Cluster: 06:    0.997 (276)
Cluster: 07:    0.427 (270)
Cluster: 08:    0.757 (240)
Cluster: 09:    0.948 (213)
Cluster: 10:    0.847 (201)
Cluster: 11:    0.903 (190)
Cluster: 12:    0.209 (147)
Cluster: 13:    0.687 (142)
Cluster: 14:    0.989 (107)
Cluster: 15:    0.749 (93)
Cluster: 16:    0.441 (102)
Cluster: 17:    0.192 (78)
Cluster: 18:    0.147 (48)


In [300]:
BM = 0
VECTOR = 1
BOTH = 2
def calculate_summary_stats(clusters_df: pd.DataFrame, method:int) -> dict[int,np.array]:
    clusters = clusters_df.groupby('cluster')
    table = {k:[] for k in range(df_test.cluster.max())}
    for cluster in clusters:
        cluster_no = cluster[0]
        cluster_df = cluster[1]
        row_count = cluster_df.shape[0]
        values = np.zeros(row_count,dtype=float)
        for row in range(row_count):
            n_score = cluster_df.n_score.iloc[row][cluster_no]
            m_score = cluster_df.scores.iloc[row][cluster_no]
            if method == BM:
                values[row] = m_score
            elif method == VECTOR:
                values[row] = n_score
            else:
                if n_score > m_score:
                    values[row] = n_score
                else:
                    values[row] = m_score
        table[cluster_no] = values
    return table


In [310]:
from scipy import stats
table_vector = calculate_summary_stats(df_test, VECTOR)
table_match = calculate_summary_stats(df_test, BM)
table_both = calculate_summary_stats(df_test, BOTH)
for cluster in table_vector:
    print(f"{cluster:02} {table_vector[cluster].mean():8.3f} {stats.sem(table_vector[cluster]):8.3f} ({table_vector[cluster].size})", end='')
    print(f"{cluster:02} {table_match[cluster].mean():8.3f} {stats.sem(table_match[cluster]):8.3f} ({table_match[cluster].size})", end='')
    print(f"{cluster:02} {table_both[cluster].mean():8.3f} {stats.sem(table_both[cluster]):8.3f} ({table_both[cluster].size})")
print("--" * 6)
for cluster in table_vector:
    print(f"{cluster:02} {((table_both[cluster].mean()/table_match[cluster].mean())-1) * 100:5.1f}")


00    0.757    0.013 (624)00    0.993    0.001 (624)00    0.996    0.001 (624)
01    0.998    0.002 (527)01    0.962    0.003 (527)01    1.000    0.000 (527)
02    0.626    0.016 (380)02    0.875    0.007 (380)02    0.916    0.006 (380)
03    0.672    0.017 (353)03    0.898    0.007 (353)03    0.938    0.006 (353)
04    0.614    0.015 (367)04    0.879    0.007 (367)04    0.897    0.007 (367)
05    0.702    0.019 (327)05    0.963    0.005 (327)05    0.975    0.004 (327)
06    0.997    0.002 (276)06    0.988    0.003 (276)06    1.000    0.000 (276)
07    0.427    0.018 (270)07    0.784    0.010 (270)07    0.823    0.010 (270)
08    0.757    0.019 (240)08    0.957    0.007 (240)08    0.971    0.006 (240)
09    0.948    0.012 (213)09    0.992    0.003 (213)09    0.994    0.002 (213)
10    0.847    0.017 (201)10    0.965    0.007 (201)10    0.984    0.004 (201)
11    0.903    0.018 (190)11    0.985    0.006 (190)11    0.989    0.005 (190)
12    0.209    0.011 (147)12    0.639    0.013 (147)